<a href="https://colab.research.google.com/github/kunamneni117/Introduction-To-Machine-Learning/blob/master/Classifications.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import Imputer,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator,TransformerMixin

In [0]:
!pip install sklearn_features

In [0]:
from sklearn_features.transformers import DataFrameSelector
from sklearn.pipeline import Pipeline,FeatureUnion